In [1]:
%matplotlib inline

In [2]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import combo
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn import preprocessing as pp
from sklearn.model_selection import KFold

In [3]:
#Import Data
os.chdir('E:\Academics\Research\Data\Experimental data\Compression_test\Results\Machined')
Master_Data = pd.read_csv('Data_Machined.csv')
Master_Data.head(5)

,Name_Horizontal,Name_Vertical,X,Y,Theta,Phi,Yield,Modulus,Max_Pore,Pore_Volume_Ratio
0,G,17,184.0,163.5,135,90,829,99,NaN,NaN
1,G,19,184.0,184.0,180,45,927,199,NaN,NaN
2,G,20,184.0,193.5,270,45,921,317,NaN,NaN
3,G,21,184.0,205.0,90,45,821,210,NaN,NaN
4,G,22,184.0,214.0,0,0,750,237,NaN,NaN


In [4]:
# Input and output columns in Dataframe
# Input = ['Logx','Logy','Cos_Theta','Sin_Theta','Cos_Phi','Sin_Phi']
Input = ['X','Y','Theta','Phi']
Properties = ['Yield','Modulus']
Output = Properties[1]

In [5]:
X = Master_Data[Input]
y = Master_Data[Output]

In [6]:
X_sc = pp.scale(X)
y_sc = pp.scale(y)

C:\Users\senth\Anaconda2\lib\site-packages\sklearn\utils\validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by the scale function.
  warnings.warn(msg, _DataConversionWarning)


In [7]:
#Split train and test data

y_p = []
y_t = []
mae = []

kf = KFold(n_splits=5)

for train, test in kf.split(X_sc, y_sc):
        X_train = X_sc[train]
        y_train = y_sc[train]
        X_test = X_sc[test]
        y_test = y_sc[test]
       
        # Fit Combo Model
        cov = combo.gp.cov.gauss(X_train.shape[1], ard = False)
        mean = combo.gp.mean.zero()
        lik = combo.gp.lik.gauss()
        gp = combo.gp.model(lik = lik,mean = mean, cov = cov)
        config = combo.misc.set_config()
        np.random.seed(1000)
        index = np.random.permutation(xrange(X_train.shape[0]))
        train_X = X_train[index,:]
        train_t = y_train[index]
        model = gp.fit(train_X, train_t, config)
        gp.prepare(train_X, train_t)

        #Predict using combo model
        y_pred = gp.get_post_fmean(train_X, X_test)
        
        y_t = np.concatenate((y_t,y_test))
        y_p = np.concatenate((y_p,y_pred))
        
        mae.append(mean_absolute_error(y_test,y_pred))


Start the initial hyper parameter searching ...
Done

Start the hyper parameter learning ...
0 -th epoch marginal likelihood 87.178891664
50 -th epoch marginal likelihood 73.9771229594
100 -th epoch marginal likelihood 65.1749099286
150 -th epoch marginal likelihood 59.2445473267
200 -th epoch marginal likelihood 55.1353789565
250 -th epoch marginal likelihood 52.2162045932
300 -th epoch marginal likelihood 50.0961119627
350 -th epoch marginal likelihood 48.525219646
400 -th epoch marginal likelihood 47.3391803974
450 -th epoch marginal likelihood 46.4271424851
500 -th epoch marginal likelihood 45.7126743691
Done



NameError: name 'y_pred' is not defined

In [ ]:
min_val = Parsed_Data[Output].min()
max_val = Parsed_Data[Output].max()
ran = max_val - min_val

In [ ]:
y_t = y_t*ran +min_val
y_p = y_p*ran+min_val

In [ ]:
plt.plot(y_t, y_p,'b.')
plt.plot(y_t,y_t,'k--')
plt.xlabel('Actual Young\'s Modulus (GPa)',size = 15)
plt.ylabel('Predicted Young\'s Modulus (Gpa)', size = 15)
plt.title('Random Forest Regression')
plt.show()

In [ ]:
mae = np.asarray(mae)

In [ ]:
mae.mean()*ran